# Project 2 - Problem 1: Publishing Data
## GIS5090 - Intro to Python Programming for GIS and Remote Sensing
### Carter Hanford
This is the Jupyter Notebook for problem 1 of project 2 in GIS5090. This problem entails publishing data, and asks us to do the following using data from the **Fragile States Index**. 

* `1) - Publish each FSI CSV file as a uniquely named feature service.`
* `2) - Create a webmap containing each feature service symbolized to indicate which countries are worse or better.`
* `3) - Describe the procress and data along the way in the Jupyter Notebook.`

**Understanding the Fragile States Index**
- The [Fragile States Index](https://fragilestatesindex.org/) is an open source dataset that ranks the fragility of the countries of the world. The FSI data is available as CSV files that ranks the courties in terms of fragility but also provides metrics on other aspects of each country's health, such as the econmony, human rights, and pliltical division.

![](https://upload.wikimedia.org/wikipedia/en/thumb/4/4b/Saint_Louis_University_logo.svg/250px-Saint_Louis_University_logo.svg.png).

#### Part 1 - Publish the CSV files
Part one asks us to take the Fragile States Index csv files and publish them as uniquely named feature services. There are 12 csv files which represents years 2006-2018 of the FSI, so publishing each csv file one at a time will make this code long. Instead, we'll use a `for loop` to iterate over the files in the designated filepath, and publishing each file using the `gis.content.add({})` function.

First, let's import the modules we'll need to pull this off:

In [1]:
# import modules
import arcpy
import arcgis 
from arcpy import env
from arcgis.gis import GIS

#path="C:\Users\Cahan\Documents\GitHub\python-programming-spring-2020\project 2\data"
#os.chdir(path)

Next we need to connect our notebook kernel to ArcGIS online: 

In [2]:
# connect to arcgis online
gis = GIS("http://slustl.maps.arcgis.com/home", "carter.hanford_SLUSTL")

Enter password: ········


Now that we've connected to ArcGIS online and imported the proper modules, we need to tell python which filepath to use. In this instance, I have all the FSI csv files in one data folder which I'll designate to an object. 

Once the filepath is designated, we can use `os.listdir()` to make sure that python is reading the files in the path:

In [7]:
# import os
import os

# set path for data
data_dir = "C:/Users/Cahan/Documents/GitHub/python-programming-spring-2020/project 2/data/"
os.chdir(data_dir)

# designate filepath object
filepath = os.listdir(data_dir)

csv_file_list = [x for x in filepath if x.endswith(".csv")]

Perfect! Now let's build the for loop. 

First, we want the loop to look in the designated filepath, and indicate that if the file ends with **.csv** (all of them), then publish them as a feature service using the `gis.content.add` function.

In [13]:
# Loop through each file and publish it as a service
for current_file in csv_file_list:
    item = gis.content.add({}, data_dir + current_file)
    if "BuildCache" not in current_file:
        published_item = item.publish(None, {"Address":"Country"}, overwrite = True)
print("Done, all csv files in path published to ArcGIS online as feature services.")

Done, all csv files in path published to ArcGIS online as feature services.


#### Part 2 - Create webmap

In this section we'll be adding the data we just published to ArcGIS online to a webmap. Let's start by creating the map object. Since this is a world map, we don't need to zoom to any specific country.

In [43]:
# create map object
map = gis.map(zoomlevel=2)

The next step is to symbolize our map and add data. We can just add the data to the map without a renderer, but in doing this its not clear what we're symbolizing. Including a renderer allows us to make each data point mean something.

Typically, we'd look at the `Rank` variable within the FGI dataset. However, this variable is character and we need a numeric variable in order to add it in the renderer. Luckily, the `Total` variable actually dictates what a country's rank is on the FGI, and it is numeric since it is calculated from the different variables that make up the index. For this reason, we'll add the `Total` variable in the renderer.

In [60]:
# set map render to class size
renderer = { "renderer": "ClassedSizeRenderer", "field_name":"Total"}

# add render and data to webmap
map.add_layer(published_item, renderer)

This part of problem 1 is pretty simple. Now all we need to do is display the webmap!

In [58]:
# display Fragile States Index Map
map

MapView(jupyter_target='notebook', layout=Layout(height='400px', width='100%'), ready=True, zoom=2.0)

Each circle symbolizes it's "rank" (based on variable `Total`) on the Fragile States Index. What we can see from this map is African countries tend to rank worse on this index, as well as some countries in South America and southern Asian countries. On the other hand, most European countries and the United States/Canada rank pretty high on the FGI. 

Now that we've created our FGI map, let's save it to ArcGIS online:

In [65]:
# add properties for online map
webmap_item_properties = {'title':'Fragile States Index',
                         'snippet':'Countries based on Index Rank',
                         'tags':['FGI', "World Map"]}

# save map 
map.save(webmap_item_properties)

<Item title:"Fragile States Index" type:Web Map owner:carter.hanford_SLUSTL>

And with that, we wrap up problem 1!